# Walmart Recruiting — Store Sales Forecasting (LightAutoML + альтернативные модели)

Цель: выполнить практическое задание курса LightAutoML.

Требования, которые закрываем в этой работе:
1. LAMA бейзлайн: минимум 2 конфигурации и выбор лучшей
2. Альтернативные решения без LAMA: PyTorch NN, RandomForest, CatBoost
3. Качественный EDA с проверкой гипотез, визуализациями, анализом пропусков и аномалий
4. Обоснованная стратегия разбиения данных без утечки
5. Self-contained пайплайны и воспроизводимый код


## 0. Установка зависимостей (Colab)

In [ ]:
!pip -q install  catboost

In [ ]:
!pip install lightautoml --no-cache-dir

## 1. Импорты и базовые настройки

In [ ]:
import os
import re
import math
import logging
import warnings
import zipfile
from dataclasses import dataclass
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")

SEED = 42
np.random.seed(SEED)


## 2. Загрузка данных

В этом ноутбуке данные читаются так, как у тебя уже сделано в Colab (csv внутри zip).

In [ ]:
zip_path = "/content/walmart-recruiting-store-sales-forecasting.zip"
extract_path = "content/walmart"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_path)

os.listdir(extract_path)

In [ ]:

DATA_DIR = "/content/content/walmart"

stores_data = pd.read_csv(f"{DATA_DIR}/stores.csv")
test_data = pd.read_csv(f"{DATA_DIR}/test.csv.zip")
train_data = pd.read_csv(f"{DATA_DIR}/train.csv.zip")
features_data = pd.read_csv(f"{DATA_DIR}/features.csv.zip")
sample_submission = pd.read_csv(f"{DATA_DIR}/sampleSubmission.csv.zip")

print(train_data.shape, test_data.shape, features_data.shape, stores_data.shape)
train_data.head()


## 3. Метрика соревнования: WMAE

Вес 5 для holiday weeks, иначе 1.

In [ ]:
def wmae(y_true: np.ndarray, y_pred: np.ndarray, is_holiday: np.ndarray) -> float:
    """
    Weighted MAE from Kaggle Walmart competition:
    holiday weeks have weight=5, non-holiday weight=1.

    Args:
        y_true: true Weekly_Sales
        y_pred: predicted Weekly_Sales
        is_holiday: binary flag (0/1) aligned with y_true rows

    Returns:
        WMAE value (float)
    """
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    w = np.where(np.asarray(is_holiday).astype(int).reshape(-1) == 1, 5.0, 1.0)
    return np.sum(w * np.abs(y_true - y_pred)) / np.sum(w)

# sanity
print("WMAE sanity:", wmae([10, 20], [12, 18], [0, 1]))


In [ ]:
from IPython.display import display

def make_submission(test_df: pd.DataFrame, preds: np.ndarray, out_path: str) -> None:
    """
    Create Kaggle submission in format Id=Store_Dept_YYYY-MM-DD.

    Note: preds must be aligned row-by-row with test_df.
    """
    preds = np.asarray(preds).reshape(-1)
    assert len(preds) == len(test_df), "preds must have same length as test_df"

    ids = (
        test_df["Store"].astype(str)
        + "_" + test_df["Dept"].astype(str)
        + "_" + test_df["Date"].astype(str)
    )
    sub = pd.DataFrame({"Id": ids, "Weekly_Sales": preds})
    sub.to_csv(out_path, index=False)


## 4. Объединение таблиц и feature engineering

Мердж: `(Store, Date, IsHoliday)` + признаки магазина.

Гипотеза: эффекты markdown и праздников лучше ловятся после добавления календарных и индикаторов пропусков markdown.

In [ ]:
def add_date_features(df: pd.DataFrame) -> pd.DataFrame:
    d = pd.to_datetime(df["Date"])
    df = df.copy()
    df["year"] = d.dt.year.astype(np.int16)
    df["month"] = d.dt.month.astype(np.int8)
    df["weekofyear"] = d.dt.isocalendar().week.astype(np.int16)
    df["dayofweek"] = d.dt.dayofweek.astype(np.int8)

    # циклические признаки
    df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12.0).astype(np.float32)
    df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12.0).astype(np.float32)
    df["week_sin"] = np.sin(2 * np.pi * df["weekofyear"] / 52.0).astype(np.float32)
    df["week_cos"] = np.cos(2 * np.pi * df["weekofyear"] / 52.0).astype(np.float32)
    return df

def build_dataset(train_df: pd.DataFrame,
                  test_df: pd.DataFrame,
                  features_df: pd.DataFrame,
                  stores_df: pd.DataFrame
                  ) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Merge train/test with features + stores, add date features.
    Handle missingness in MarkDown columns by:
    - add isna flags (model can learn 'missingness pattern')
    - fill NaNs with 0 (common practice for these promo-related features)
    """
    train = train_df.merge(features_df, on=["Store", "Date", "IsHoliday"], how="left").merge(stores_df, on="Store", how="left")
    test = test_df.merge(features_df, on=["Store", "Date", "IsHoliday"], how="left").merge(stores_df, on="Store", how="left")

    train = add_date_features(train)
    test = add_date_features(test)

    # MarkDown columns + флаги пропусков
    md_cols = [c for c in train.columns if re.fullmatch(r"MarkDown[1-5]", c)]
    for c in md_cols:
        train[c + "_isna"] = train[c].isna().astype(np.int8)
        test[c + "_isna"] = test[c].isna().astype(np.int8)
        train[c] = train[c].fillna(0.0)
        test[c] = test[c].fillna(0.0)

    train["IsHoliday"] = train["IsHoliday"].astype(np.int8)
    test["IsHoliday"] = test["IsHoliday"].astype(np.int8)

    # редкие пропуски в базовых фичах
    cont_fill = ["Temperature","Fuel_Price","CPI","Unemployment","Size"]
    for c in cont_fill:
        train[c] = train[c].fillna(train[c].median())
        test[c] = test[c].fillna(train[c].median())

    return train, test

df_train, df_test = build_dataset(train_data, test_data, features_data, stores_data)
df_train.shape, df_test.shape


## 5. EDA

### 5.1 Анализ таргета

Целевая переменная `Weekly_Sales` имеет распределение с тяжёлым хвостом и выраженными выбросами,
которые в основном связаны с праздничными неделями и сезонными всплесками спроса.
В связи с этим метрики, чувствительные к выбросам (например RMSE), могут быть нестабильны,
поэтому в работе используется MAE.

Также продажи имеют выраженную временную структуру и сезонность,
что требует временного разделения данных для корректной оценки качества модели
и предотвращения утечки данных.


In [ ]:
target = df_train["Weekly_Sales"].astype(float)

print("Target describe:")
display(target.describe(percentiles=[0.01,0.05,0.5,0.95,0.99]))

# распределение
plt.figure(figsize=(10,4))
plt.hist(np.clip(target, 0, target.quantile(0.99)), bins=80)
plt.title("Weekly_Sales distribution (clipped at 99p)")
plt.show()

# аномалии: отрицательные продажи (есть в датасете)
neg_cnt = (target < 0).sum()
print("Negative sales count:", int(neg_cnt), "share:", float(neg_cnt/len(target)))

# динамика во времени (агрегация по дате)
tmp = df_train.groupby("Date", as_index=False).agg(
    sales_sum=("Weekly_Sales","sum"),
    is_holiday=("IsHoliday","max")
)
tmp["Date"] = pd.to_datetime(tmp["Date"])

plt.figure(figsize=(12,4))
plt.plot(tmp["Date"], tmp["sales_sum"])
plt.title("Total sales over time (sum over all stores/depts)")
plt.show()

print("Holiday weeks vs non-holiday weeks (sum sales):")
display(tmp.groupby("is_holiday")["sales_sum"].describe())


### 5.2 Анализ признаков

### Типы признаков

Числовые признаки:
- Temperature
- Fuel_Price
- CPI
- Unemployment

Категориальные признаки:
- Store
- Dept
- IsHoliday

Временной признак:
- Date


Пропуски в макроэкономических показателях связаны с неполнотой внешней отчётности.
Для обработки пропусков использовалась медианная импутация,
что позволяет избежать утечки информации из будущего.


In [ ]:
# Типизация (базово)
cat_cols = ["Store", "Dept", "Type"]
date_cols = ["Date"]
# числовые: всё остальное кроме таргета
num_cols = [c for c in df_train.columns if c not in (cat_cols + date_cols + ["Weekly_Sales"])]

print("Categorical:", cat_cols)
print("Date:", date_cols)
print("Numeric count:", len(num_cols))

# пропуски
na = df_train[num_cols].isna().mean().sort_values(ascending=False)
display(na.head(15))

plt.figure(figsize=(10,4))
plt.bar(na.index[:15], na.values[:15])
plt.xticks(rotation=45, ha="right")
plt.title("Top-15 missingness among numeric features (train)")
plt.show()

# зависимости: Temperature vs Weekly_Sales (пример)
plt.figure(figsize=(6,4))
sample = df_train.sample(20000, random_state=SEED)
plt.scatter(sample["Temperature"], sample["Weekly_Sales"], s=2)
plt.title("Temperature vs Weekly_Sales (sample)")
plt.xlabel("Temperature")
plt.ylabel("Weekly_Sales")
plt.show()

# корреляции с таргетом (на подвыборке, чтобы быстрее)
sample = df_train.sample(20000, random_state=SEED)

corr_df = (
    sample[num_cols]
    .corrwith(sample["Weekly_Sales"], numeric_only=True)
    .sort_values(key=np.abs, ascending=False)
)
display(corr_df.head(15))

plt.figure(figsize=(10,4))
plt.bar(corr_df.head(15).index, corr_df.head(15).values)
plt.xticks(rotation=45, ha="right")
plt.title("Top-15 abs correlations with target (sample)")
plt.show()


## 6. Стратегия разбиения данных

Обоснование: это временной ряд по неделям. Чтобы избежать утечки, используем **chronological split**: последние `val_weeks` недель валидация.

In [ ]:
def chronological_split(train_df: pd.DataFrame,
                        val_weeks: int = 8
                        ) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Time-based split to avoid leakage: validation contains the most recent weeks.
    We split by Date (weekly granularity in dataset).
    """
    dates = np.array(sorted(train_df["Date"].unique()))
    cut = dates[-val_weeks] if len(dates) > val_weeks else dates[int(len(dates)*0.8)]
    tr = train_df[train_df["Date"] < cut].copy()
    va = train_df[train_df["Date"] >= cut].copy()
    return tr, va

train_tr, train_va = chronological_split(df_train, val_weeks=8)
print(train_tr["Date"].min(), train_tr["Date"].max(), train_va["Date"].min(), train_va["Date"].max())
print(train_tr.shape, train_va.shape)


## 7. Бейзлайн на LightAutoML (2 конфигурации)

Важно: LAMA оптимизирует MAE, а итоговую оценку мы считаем WMAE на валидации.

Конфигурации:
1. Быстрый бейзлайн (меньше timeout)
2. Усиленный (больше timeout, больше фолдов)


In [ ]:
from lightautoml.tasks import Task
from lightautoml.automl.presets.tabular_presets import TabularAutoML

# подготовка данных для LAMA
lama_tr = train_tr.copy()
lama_va = train_va.copy()

# для LAMA target в отдельной колонке
target_col = "Weekly_Sales"

# 1) baseline config
task = Task("reg", metric="mae", greater_is_better=False)

automl_lama_1 = TabularAutoML(
    task=task,
    timeout=300,
    cpu_limit=4,
    general_params={"use_algos": [["lgb", "lgb_tuned"]]},
    reader_params={"random_state": SEED, "cv": 3},
)

oof_1 = automl_lama_1.fit_predict(lama_tr, roles={"target": target_col})
pred_va_1 = automl_lama_1.predict(lama_va).data[:, 0]

lama_wmae_1 = wmae(lama_va[target_col].values, pred_va_1, lama_va["IsHoliday"].values)
print("LAMA #1 val WMAE:", lama_wmae_1)

# 2) stronger config
automl_lama_2 = TabularAutoML(
    task=task,
    timeout=900,
    cpu_limit=4,
    general_params={"use_algos": [["lgb", "lgb_tuned", "cb", "linear_l2"]]},
    reader_params={"random_state": SEED, "cv": 5},
)

oof_2 = automl_lama_2.fit_predict(lama_tr, roles={"target": target_col})
pred_va_2 = automl_lama_2.predict(lama_va).data[:, 0]

lama_wmae_2 = wmae(lama_va[target_col].values, pred_va_2, lama_va["IsHoliday"].values)
print("LAMA #2 val WMAE:", lama_wmae_2)

best_lama = automl_lama_1 if lama_wmae_1 <= lama_wmae_2 else automl_lama_2
best_lama_name = "LAMA_1" if best_lama is automl_lama_1 else "LAMA_2"
print("Best LAMA:", best_lama_name)


## 8. Альтернативные решения без LAMA

Ниже 3 модели: RandomForest, CatBoost, PyTorch NN.

Общее: одна и та же логика подготовки признаков и одинаковая оценка WMAE.

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor


### 8.1 RandomForest

Примечание: у RF плохо масштабируется one-hot на Store/Dept, поэтому делаем:
- Type: one-hot
- Store/Dept: частотное кодирование
- числовые: как есть

Это не лучший метод, но это отдельный пайплайн, который закрывает требование проекта.

In [ ]:
def add_frequency_encoding(
    tr: pd.DataFrame,
    va: pd.DataFrame,
    cols: List[str]
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:

    """
    Add frequency encoding for categorical columns.

    For each column in `cols`, computes category frequencies on the training
    split only (to avoid data leakage) and maps them to both train and
    validation datasets.

    Missing categories in validation are filled with 0.

    Args:
        tr: Training dataframe.
        va: Validation dataframe.
        cols: List of categorical column names to encode.

    Returns:
        Tuple of (train_df_with_freq, val_df_with_freq).
    """
    tr = tr.copy()
    va = va.copy()
    for c in cols:
        freq = tr[c].value_counts(dropna=False)
        tr[c + "_freq"] = tr[c].map(freq).astype(np.float32)
        va[c + "_freq"] = va[c].map(freq).fillna(0).astype(np.float32)
    return tr, va

rf_tr, rf_va = add_frequency_encoding(train_tr, train_va, ["Store","Dept"])

rf_target = rf_tr["Weekly_Sales"].values
rf_is_hol = rf_va["IsHoliday"].values

rf_num = [c for c in rf_tr.columns if c not in ["Weekly_Sales","Date","Type","Store","Dept"]]
rf_cat = ["Type"]

preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), rf_num),
        ("cat", OneHotEncoder(handle_unknown="ignore"), rf_cat),
    ],
    remainder="drop",
)

rf_model = RandomForestRegressor(
    n_estimators=300,
    random_state=SEED,
    n_jobs=-1,
    max_depth=None,
)

rf_pipe = Pipeline([
    ("prep", preprocess),
    ("model", rf_model),
])

rf_pipe.fit(rf_tr, rf_target)
rf_pred = rf_pipe.predict(rf_va)

rf_wmae = wmae(train_va["Weekly_Sales"].values, rf_pred, rf_is_hol)
print("RF val WMAE:", rf_wmae)


### 8.2 CatBoost

CatBoost нативно обрабатывает категориальные фичи `Store`, `Dept`, `Type`, что обычно дает хороший результат на этом датасете.


In [ ]:
from catboost import CatBoostRegressor

def add_date_parts(df):
    """
    Добавляет календарные признаки, извлечённые из колонки `Date`.

    Функция преобразует столбец `Date` в формат datetime и на его основе
    создаёт набор стандартных временных признаков, которые часто используются
    в задачах прогнозирования временных рядов и табличных моделях.

    Добавляемые колонки:
    - Year        : год (int)
    - Week        : номер недели по ISO-календарю (int)
    - Month       : месяц (1–12)
    - Day         : день месяца (1–31)
    - DayOfWeek   : день недели (0 = понедельник, 6 = воскресенье)

    Параметры
    ---------
    df : pandas.DataFrame
        DataFrame, содержащий колонку `Date`.
        Колонка `Date` должна быть приводима к datetime через `pd.to_datetime`.

    Возвращает
    ----------
    pandas.DataFrame
        Тот же DataFrame, дополненный календарными признаками.

    Примечания
    ----------
    - Номер недели считается по ISO-календарю (ISO-8601).
    - Функция модифицирует входной DataFrame inplace и возвращает его же.
    """
    d = pd.to_datetime(df["Date"])
    df["Year"] = d.dt.year.astype(int)
    df["Week"] = d.dt.isocalendar().week.astype(int)
    df["Month"] = d.dt.month.astype(int)
    df["Day"] = d.dt.day.astype(int)
    df["DayOfWeek"] = d.dt.dayofweek.astype(int)
    return df

train_tr = add_date_parts(train_tr.copy())
train_va = add_date_parts(train_va.copy())

cb_features = [c for c in train_tr.columns if c not in ["Weekly_Sales", "Date"]]
cb_cat = ["Store", "Dept", "Type"]

cb_train = train_tr[cb_features].copy()
cb_valid = train_va[cb_features].copy()


cb = CatBoostRegressor(
    loss_function="MAE",
    iterations=5000,
    learning_rate=0.03,
    depth=10,
    random_seed=SEED,
    eval_metric="MAE",
    early_stopping_rounds=200,
    verbose=200,
)

cb.fit(
    cb_train,
    train_tr["Weekly_Sales"],
    eval_set=(cb_valid, train_va["Weekly_Sales"]),
    cat_features=cb_cat,
)

cb_pred = cb.predict(cb_valid)
cb_wmae = wmae(train_va["Weekly_Sales"].values, cb_pred, train_va["IsHoliday"].values)
print("CatBoost val WMAE:", cb_wmae)


### 8.3 PyTorch Neural Network

Эмбеддинги Store/Dept/Type + MLP. Лосс = Weighted MAE (совпадает с метрикой).


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

def seed_everything(seed: int = 42) -> None:
    """
    Фиксирует сиды (random/NumPy/PyTorch) для воспроизводимости экспериментов.

    Что делает:
    - задаёт seed для Python `random`, NumPy и PyTorch;
    - задаёт seed для всех CUDA-девайсов (если есть);
    - включает детерминированный режим cuDNN (где это возможно) и отключает авто-бенчмарк,
      чтобы результаты меньше зависели от недетерминированных оптимизаций.

    Параметры
    ---------
    seed : int, по умолчанию 42
        Значение сида.

    Возвращает
    ----------
    None
    """
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def weighted_mae_torch(pred: torch.Tensor, target: torch.Tensor, weight: torch.Tensor) -> torch.Tensor:
    """
    Считает взвешенную MAE (WMAE) в PyTorch.

    Формула:
        WMAE = sum_i w_i * |pred_i - target_i| / sum_i w_i

    Все входные тензоры приводятся к 1D (flatten). Сумма весов защищена от деления на ноль.

    Параметры
    ---------
    pred : torch.Tensor
        Предсказания модели (любой формы; будет расплющен в 1D).
    target : torch.Tensor
        Истинные значения (любой формы; будет расплющен в 1D).
    weight : torch.Tensor
        Веса объектов (любой формы; будет расплющен в 1D).

    Возвращает
    ----------
    torch.Tensor
        Скалярный тензор WMAE.
    """
    pred = pred.view(-1)
    target = target.view(-1)
    weight = weight.view(-1)
    return (weight * (pred - target).abs()).sum() / weight.sum().clamp_min(1.0)

@torch.no_grad()
def eval_wmae_torch(model: nn.Module, loader: DataLoader, device: torch.device) -> float:
    """
    Оценивает модель на DataLoader по метрике WMAE (взвешенная MAE).

    Важно: метрика агрегируется по всему датасету через суммирование числителя/знаменателя,
    чтобы не возникал перекос из-за усреднения по батчам.

    Ожидаемый формат батча из loader:
        (x_cat, x_cont, y, w)
    где:
        x_cat  : LongTensor [B, n_cat]
        x_cont : FloatTensor [B, n_cont]
        y      : FloatTensor [B] или [B, 1]
        w      : FloatTensor [B] или [B, 1]

    Параметры
    ---------
    model : nn.Module
        Модель с сигнатурой model(x_cat, x_cont) -> предсказания.
    loader : DataLoader
        Валид/тест DataLoader, который отдаёт батчи (x_cat, x_cont, y, w).
    device : torch.device
        Устройство, на котором считать инференс (cpu/cuda).

    Возвращает
    ----------
    float
        Значение WMAE по всему набору данных.
    """
    model.eval()
    num, den = 0.0, 0.0
    for x_cat, x_cont, y, w in loader:
        x_cat = x_cat.to(device)
        x_cont = x_cont.to(device)
        y = y.to(device)
        w = w.to(device)
        p = model(x_cat, x_cont).view(-1)
        num += float((w.view(-1) * (p - y.view(-1)).abs()).sum().cpu())
        den += float(w.sum().cpu())
    return num / max(den, 1e-12)

class WalmartDS(Dataset):
    """
    Dataset для табличной задачи Walmart Weekly Sales.

    Хранит:
    - категориальные признаки (индексы) в int64 (long) тензорах;
    - непрерывные признаки в float32 тензорах;
    - целевую переменную `Weekly_Sales` в float32;
    - веса объектов (например, праздники x5) в float32.

    Параметры
    ---------
    x_cat : np.ndarray
        Категориальные признаки формы [N, n_cat], уже закодированные индексами.
    x_cont : np.ndarray
        Непрерывные признаки формы [N, n_cont].
    y : np.ndarray
        Таргет формы [N] или [N, 1].
    w : np.ndarray
        Веса объектов формы [N] или [N, 1].

    Возвращаемое значение __getitem__
    -------------------------------
    tuple(torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor)
        (x_cat[i], x_cont[i], y[i], w[i])
    """
    def __init__(self, x_cat, x_cont, y, w):
        self.x_cat = torch.from_numpy(x_cat).long()
        self.x_cont = torch.from_numpy(x_cont).float()
        self.y = torch.from_numpy(y).float()
        self.w = torch.from_numpy(w).float()
    def __len__(self):
        """Возвращает количество объектов в датасете."""
        return len(self.y)
    def __getitem__(self, i):
        """
        Возвращает один объект по индексу.

        Параметры
        ---------
        i : int
            Индекс объекта.

        Возвращает
        ----------
        tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]
            (x_cat, x_cont, y, w) для данного индекса.
        """
        return self.x_cat[i], self.x_cont[i], self.y[i], self.w[i]

class TabularNN(nn.Module):
    """
    Нейросеть для табличных данных: эмбеддинги категорий + MLP.

    Категориальные признаки:
    - Store -> эмбеддинг размерности 8
    - Dept  -> эмбеддинг размерности 16
    - Type  -> эмбеддинг размерности 4

    Далее эмбеддинги конкатенируются с непрерывными признаками и подаются в MLP.

    Параметры
    ---------
    n_store : int
        Количество уникальных Store (размер словаря эмбеддинга).
    n_dept : int
        Количество уникальных Dept (размер словаря эмбеддинга).
    n_type : int
        Количество уникальных Type (размер словаря эмбеддинга).
    cont_dim : int
        Число непрерывных признаков.
    hidden : int, по умолчанию 256
        Ширина скрытых слоёв MLP.
    drop : float, по умолчанию 0.1
        Dropout в MLP.

    Вход forward
    ------------
    x_cat : torch.LongTensor
        Тензор формы [B, 3] со столбцами: [store_idx, dept_idx, type_idx].
    x_cont : torch.FloatTensor
        Тензор формы [B, cont_dim] с непрерывными признаками.

    Возвращает
    ----------
    torch.Tensor
        Предсказания формы [B, 1].
    """
    def __init__(self, n_store, n_dept, n_type, cont_dim, hidden=256, drop=0.1):
        super().__init__()
        self.emb_store = nn.Embedding(n_store, 8)
        self.emb_dept = nn.Embedding(n_dept, 16)
        self.emb_type = nn.Embedding(n_type, 4)
        in_dim = 8 + 16 + 4 + cont_dim
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Dropout(drop),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Dropout(drop),
            nn.Linear(hidden, 1),
        )
    def forward(self, x_cat, x_cont):
        """
        Прямой проход: эмбеддинги -> конкатенация -> MLP.

        Параметры
        ---------
        x_cat : torch.LongTensor
            Категориальные индексы, форма [B, 3].
        x_cont : torch.FloatTensor
            Непрерывные признаки, форма [B, cont_dim].

        Возвращает
        ----------
        torch.Tensor
            Предсказания, форма [B, 1].
        """
        s = self.emb_store(x_cat[:,0])
        d = self.emb_dept(x_cat[:,1])
        t = self.emb_type(x_cat[:,2])
        x = torch.cat([s,d,t,x_cont], dim=1)
        return self.mlp(x)

def build_encoders(train_df: pd.DataFrame, test_df: pd.DataFrame):
    """
    Строит отображения (энкодеры) категорий Store/Dept/Type в индексы.

    Маппинги строятся по объединению train + test, чтобы:
    - словари категорий были стабильны,
    - на тесте не появлялись "неизвестные" категории.

    Параметры
    ---------
    train_df : pd.DataFrame
        DataFrame с колонками 'Store', 'Dept', 'Type' (обучающая часть).
    test_df : pd.DataFrame
        DataFrame с колонками 'Store', 'Dept', 'Type' (тест).

    Возвращает
    ----------
    tuple[dict, dict, dict]
        store2idx, dept2idx, type2idx:
        словари вида {значение_категории -> индекс}.
    """
    all_store = pd.concat([train_df["Store"], test_df["Store"]]).unique()
    all_dept = pd.concat([train_df["Dept"], test_df["Dept"]]).unique()
    all_type = pd.concat([train_df["Type"], test_df["Type"]]).astype(str).unique()

    store2idx = {int(v): i for i, v in enumerate(sorted(all_store))}
    dept2idx = {int(v): i for i, v in enumerate(sorted(all_dept))}
    type2idx = {str(v): i for i, v in enumerate(sorted(all_type))}
    return store2idx, dept2idx, type2idx

def make_nn_arrays(df: pd.DataFrame, store2idx, dept2idx, type2idx, cont_cols: List[str]):
    """
    Преобразует DataFrame в NumPy-массивы (x_cat, x_cont) для табличной NN.

    Формирование x_cat идёт в фиксированном порядке:
        [Store_idx, Dept_idx, Type_idx]
    x_cont формируется выборкой колонок `cont_cols` и приводится к float32.

    Параметры
    ---------
    df : pd.DataFrame
        Входной DataFrame, содержащий 'Store', 'Dept', 'Type' и континуальные признаки.
    store2idx : dict
        Словарь {Store -> индекс}.
    dept2idx : dict
        Словарь {Dept -> индекс}.
    type2idx : dict
        Словарь {Type(str) -> индекс}.
    cont_cols : list[str]
        Список колонок непрерывных признаков.

    Возвращает
    ----------
    tuple[np.ndarray, np.ndarray]
        x_cat : np.ndarray, dtype=int64, форма [N, 3]
        x_cont: np.ndarray, dtype=float32, форма [N, len(cont_cols)]
    """
    x_cat = np.stack([
        df["Store"].map(store2idx).astype(np.int64).values,
        df["Dept"].map(dept2idx).astype(np.int64).values,
        df["Type"].astype(str).map(type2idx).astype(np.int64).values,
    ], axis=1)
    x_cont = df[cont_cols].astype(np.float32).values
    return x_cat, x_cont

def standardize(train_x, val_x, test_x):
    """
    Стандартизирует непрерывные признаки по статистикам train.

    Для каждого признака:
        x' = (x - mean_train) / std_train

    Если std слишком мал (признак почти константный), std заменяется на 1.0,
    чтобы избежать деления на почти ноль.

    Параметры
    ---------
    train_x : np.ndarray
        Непрерывные признаки train, форма [N_train, D].
    val_x : np.ndarray
        Непрерывные признаки val, форма [N_val, D].
    test_x : np.ndarray
        Непрерывные признаки test (или любой другой сплит), форма [N_test, D].

    Возвращает
    ----------
    tuple[np.ndarray, np.ndarray, np.ndarray]
        (train_x_std, val_x_std, test_x_std) тех же форм, что и входы.
    """
    m = train_x.mean(axis=0, keepdims=True)
    s = train_x.std(axis=0, keepdims=True)
    s = np.where(s < 1e-6, 1.0, s)
    return (train_x-m)/s, (val_x-m)/s, (test_x-m)/s

# NN feature set
md_cols = [c for c in df_train.columns if re.fullmatch(r"MarkDown[1-5]", c)]
md_na_cols = [c + "_isna" for c in md_cols]
nn_cont = [
    "IsHoliday",
    "Temperature","Fuel_Price","CPI","Unemployment",
    "Size",
    "year","month","weekofyear","dayofweek",
    "month_sin","month_cos","week_sin","week_cos",
] + md_cols + md_na_cols

store2idx, dept2idx, type2idx = build_encoders(train_tr, df_test)

x_cat_tr, x_cont_tr = make_nn_arrays(train_tr, store2idx, dept2idx, type2idx, nn_cont)
x_cat_va, x_cont_va = make_nn_arrays(train_va, store2idx, dept2idx, type2idx, nn_cont)

x_cont_tr, x_cont_va, _ = standardize(x_cont_tr, x_cont_va, x_cont_va)

y_tr = train_tr["Weekly_Sales"].astype(np.float32).values
y_va = train_va["Weekly_Sales"].astype(np.float32).values

w_tr = np.where(train_tr["IsHoliday"].values == 1, 5.0, 1.0).astype(np.float32)
w_va = np.where(train_va["IsHoliday"].values == 1, 5.0, 1.0).astype(np.float32)

ds_tr = WalmartDS(x_cat_tr, x_cont_tr, y_tr, w_tr)
ds_va = WalmartDS(x_cat_va, x_cont_va, y_va, w_va)

dl_tr = DataLoader(ds_tr, batch_size=4096, shuffle=True, num_workers=0)
dl_va = DataLoader(ds_va, batch_size=4096, shuffle=False, num_workers=0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed_everything(SEED)

model = TabularNN(
    n_store=len(store2idx),
    n_dept=len(dept2idx),
    n_type=len(type2idx),
    cont_dim=x_cont_tr.shape[1],
).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=2e-3, weight_decay=1e-4)

best = 1e18
best_state = None
patience = 4
bad = 0

for ep in range(1, 30+1):
    model.train()
    losses=[]
    for x_cat, x_cont, y, w in dl_tr:
        x_cat = x_cat.to(device)
        x_cont = x_cont.to(device)
        y = y.to(device)
        w = w.to(device)

        p = model(x_cat, x_cont).view(-1)
        loss = weighted_mae_torch(p, y.view(-1), w.view(-1))

        opt.zero_grad(set_to_none=True)
        loss.backward()
        opt.step()
        losses.append(float(loss.detach().cpu()))
    val = eval_wmae_torch(model, dl_va, device)
    print(f"ep={ep:02d} train_loss={np.mean(losses):.5f} val_WMAE={val:.5f}")

    if val < best - 1e-4:
        best = val
        best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        bad = 0
    else:
        bad += 1
        if bad >= patience:
            break

if best_state is not None:
    model.load_state_dict(best_state)

nn_pred_va = []
with torch.no_grad():
    model.eval()
    for x_cat, x_cont, y, w in dl_va:
        p = model(x_cat.to(device), x_cont.to(device)).view(-1).cpu().numpy()
        nn_pred_va.append(p)
nn_pred_va = np.concatenate(nn_pred_va)

nn_wmae = wmae(train_va["Weekly_Sales"].values, nn_pred_va, train_va["IsHoliday"].values)
print("NN val WMAE:", nn_wmae)


## 9. Сравнение моделей и выбор лучшей

In [ ]:
results = pd.DataFrame({
    "model": ["LAMA_1", "LAMA_2", "RandomForest", "CatBoost", "NeuralNet"],
    "val_WMAE": [lama_wmae_1, lama_wmae_2, rf_wmae, cb_wmae, nn_wmae],
}).sort_values("val_WMAE")

display(results)


## 10. Обучение на всем train и формирование submission

берём лучшую из моделей и обучаем на full train, затем предсказываем test.

В этом блоке показан пример для LAMA_2 (лучший).


In [ ]:
BEST_MODEL = "LAMA_2"  # "CatBoost", "LAMA", "NN", "RF"
os.makedirs("/content/out", exist_ok=True)

if BEST_MODEL == "CatBoost":
    full_features = [c for c in df_train.columns if c not in ["Weekly_Sales"]]
    cb_full = CatBoostRegressor(
        loss_function="MAE",
        iterations=3000,
        learning_rate=0.03,
        depth=10,
        random_seed=SEED,
        verbose=200,
    )
    cb_full.fit(df_train[full_features], df_train["Weekly_Sales"], cat_features=["Store","Dept","Type"])
    test_pred = cb_full.predict(df_test[full_features])

    out_path = "/content/out/submission_catboost.csv"
    make_submission(df_test, test_pred, out_path)
    print("Wrote:", out_path)

elif BEST_MODEL == "LAMA":
    # использовать best_lama из секции 7
    test_pred = best_lama.predict(df_test).data[:, 0]
    out_path = "/content/out/submission_lama.csv"
    make_submission(df_test, test_pred, out_path)
    print("Wrote:", out_path)

else:
    print("Для RF и NN сабмит делается аналогично: обучаешь на df_train, предсказываешь df_test.")


In [ ]:
BEST_MODEL = "LAMA"  # "CatBoost", "LAMA", "NN", "RF"
os.makedirs("/content/out", exist_ok=True)

if BEST_MODEL == "CatBoost":
    full_features = [c for c in df_train.columns if c not in ["Weekly_Sales"]]
    cb_full = CatBoostRegressor(
        loss_function="MAE",
        iterations=3000,
        learning_rate=0.03,
        depth=10,
        random_seed=SEED,
        verbose=200,
    )
    cb_full.fit(df_train[full_features], df_train["Weekly_Sales"], cat_features=["Store","Dept","Type"])
    test_pred = cb_full.predict(df_test[full_features])

    out_path = "/content/out/submission_catboost.csv"
    make_submission(df_test, test_pred, out_path)
    print("Wrote:", out_path)

elif BEST_MODEL == "LAMA":
    # использовать best_lama из секции 7
    test_pred = best_lama.predict(df_test).data[:, 0]
    out_path = "/content/out/submission_lama.csv"
    make_submission(df_test, test_pred, out_path)
    print("Wrote:", out_path)

else:
    print("Для RF и NN сабмит делается аналогично: обучаешь на df_train, предсказываешь df_test.")


## 11. Доп альтернативное решение с лучшим скором, чем LLAMA c другим FE и другими моделями

In [ ]:
import time
import logging
from datetime import datetime

from sklearn import ensemble
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# 0) Fast hyperparameter grid
param_grid_fast = {
    "model__n_estimators": [200, 300, 400],
    "model__max_depth": [12, 16, 20],
    "model__min_samples_leaf": [5, 10, 20, 50],
    "model__max_features": ["sqrt", 0.5],
}

# 1) Логирование (в консоль + файл)
log_path = f"rf_tuning_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[logging.FileHandler(log_path), logging.StreamHandler()],
)

logging.info("Start RF tuning")
logging.info("Train full shape: %s | Valid shape: %s", rf_tr.shape, rf_va.shape)


# 2) Subsample для ускорения подбора
tune_n = 120_000
rf_tr_tune = rf_tr.sample(n=min(tune_n, len(rf_tr)), random_state=SEED)
rf_target_tune = rf_tr_tune["Weekly_Sales"].values

logging.info("Tune sample shape: %s", rf_tr_tune.shape)


# 3) Time-aware CV
tscv = TimeSeriesSplit(n_splits=2)

# 4) RandomizedSearch
search = RandomizedSearchCV(
    estimator=rf_pipe,
    param_distributions=param_grid_fast,
    n_iter=8,
    scoring="neg_mean_absolute_error",
    cv=tscv,
    random_state=SEED,
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
)

t0 = time.time()
search.fit(rf_tr_tune, rf_target_tune)
logging.info("Search finished in %.1fs", time.time() - t0)

logging.info("Best params: %s", search.best_params_)
logging.info("Best CV MAE: %.6f", -search.best_score_)

# 5) Оценка на holdout (WMAE)
best_rf_pipe = search.best_estimator_
rf_pred_tuned = best_rf_pipe.predict(rf_va)

rf_wmae_tuned = wmae(
    train_va["Weekly_Sales"].values,
    rf_pred_tuned,
    rf_is_hol
)

print("RF tuned val WMAE:", rf_wmae_tuned)
print("Best params:", search.best_params_)
print("Log file:", log_path)



In [ ]:
def type_conversion_full(df: pd.DataFrame) -> pd.DataFrame:
    """
    Преобразует категориальный признак `Type` в числовое представление.

    Выполняет детерминированное отображение строковых значений типа магазина
    в числовые коды, пригодные для использования в табличных моделях.

    Правило преобразования:
    - "A" → 3
    - "B" → 2
    - все остальные значения → 1

    Параметры
    ---------
    df : pandas.DataFrame
        DataFrame, содержащий колонку `Type`.

    Возвращает
    ----------
    pandas.DataFrame
        Копию входного DataFrame, в которой колонка `Type`
        заменена на числовое значение.

    Примечания
    ----------
    - Функция создаёт копию DataFrame и не модифицирует входной объект inplace.
    - Предполагается, что множество значений `Type` ограничено
      ("A", "B" и прочие категории).
    """
    df = df.copy()
    df["Type"] = df["Type"].apply(lambda x: 3 if x == "A" else (2 if x == "B" else 1))
    return df

feature_store = features_data.merge(stores_data, how="inner", on="Store")

feature_store["Date"] = pd.to_datetime(feature_store["Date"])
train_data["Date"] = pd.to_datetime(train_data["Date"])
test_data["Date"] = pd.to_datetime(test_data["Date"])

feature_store["Week"] = feature_store["Date"].dt.isocalendar().week
feature_store["Year"] = feature_store["Date"].dt.year
feature_store["Day"] = feature_store["Date"].dt.day

train_df = (
    train_data.merge(feature_store, how="inner", on=["Store", "Date", "IsHoliday"])
    .sort_values(by=["Store", "Dept", "Date"])
    .reset_index(drop=True)
)

test_df = (
    test_data.merge(feature_store, how="inner", on=["Store", "Date", "IsHoliday"])
    .sort_values(by=["Store", "Dept", "Date"])
    .reset_index(drop=True)
)

train_df.loc[(train_df.Year == 2010) & (train_df.Week == 13), "IsHoliday"] = True
train_df.loc[(train_df.Year == 2011) & (train_df.Week == 16), "IsHoliday"] = True
train_df.loc[(train_df.Year == 2012) & (train_df.Week == 14), "IsHoliday"] = True
test_df.loc[(test_df.Year == 2013) & (test_df.Week == 13), "IsHoliday"] = True

train_df.loc[(train_df.Year == 2010) & (train_df.Week == 18), "IsHoliday"] = True
train_df.loc[(train_df.Year == 2011) & (train_df.Week == 18), "IsHoliday"] = True
train_df.loc[(train_df.Year == 2012) & (train_df.Week == 18), "IsHoliday"] = True
test_df.loc[(test_df.Year == 2013) & (test_df.Week == 18), "IsHoliday"] = True

train_df.loc[(train_df.Year == 2010) & (train_df.Week == 26), "IsHoliday"] = True
train_df.loc[(train_df.Year == 2011) & (train_df.Week == 26), "IsHoliday"] = True
train_df.loc[(train_df.Year == 2012) & (train_df.Week == 27), "IsHoliday"] = True
test_df.loc[(test_df.Year == 2013) & (test_df.Week == 27), "IsHoliday"] = True

train_df = type_conversion_full(train_df)
test_df = type_conversion_full(test_df)

feature_cols = ["Store", "Dept", "IsHoliday", "Size", "Type", "Week", "Year", "Day"]

X = train_df[feature_cols].copy()
y = train_df["Weekly_Sales"].copy()

X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    random_state=0,
    test_size=0.1,
)

rf = RandomForestRegressor(
    n_estimators=search.best_params_["model__n_estimators"],
    max_depth=search.best_params_["model__max_depth"],
    min_samples_leaf=search.best_params_["model__min_samples_leaf"],
    max_features=search.best_params_["model__max_features"],
    random_state=SEED,
    n_jobs=-1,
)
rf.fit(X_train, y_train.values.ravel())

etr = ensemble.ExtraTreesRegressor(
    bootstrap=True,
    random_state=0,
)
etr.fit(X_train, y_train.values.ravel())

X_test = test_df[feature_cols].copy()
pred_rf = rf.predict(X_test)
pred_etr = etr.predict(X_test)
avg_preds = (pred_rf + pred_etr) / 2

test_strip = test_df[["Store", "Dept", "Date", "Week", "Year"]].copy()
test_strip["Weekly_Sales"] = avg_preds


def week_51_adj(row: pd.Series) -> float:
    compareval = test_strip[
        (test_strip["Store"] == row.Store)
        & (test_strip["Dept"] == row.Dept)
        & (test_strip["Week"] == 52)
    ]
    if compareval.empty:
        return row.Weekly_Sales
    if row.Weekly_Sales > 1.5 * compareval.Weekly_Sales.median():
        return row.Weekly_Sales * 0.85
    return row.Weekly_Sales


def week_52_adj(row: pd.Series) -> float:
    compareval = test_strip[
        (test_strip["Store"] == row.Store)
        & (test_strip["Dept"] == row.Dept)
        & (test_strip["Week"] == 51)
    ]
    if compareval.empty:
        return row.Weekly_Sales
    if row.Weekly_Sales * 1.275 < compareval.Weekly_Sales.median():
        return row.Weekly_Sales * 1.2
    return row.Weekly_Sales


test_strip["Weekly_Sales"] = test_strip.apply(
    lambda row: week_51_adj(row) if row.Week == 51 else row.Weekly_Sales,
    axis=1,
)
test_strip["Weekly_Sales"] = test_strip.apply(
    lambda row: week_52_adj(row) if row.Week == 52 else row.Weekly_Sales,
    axis=1,
)

sample_submission["Weekly_Sales"] = test_strip["Weekly_Sales"]
sample_submission.to_csv("submission.csv", index=False)


In [ ]:
# predictions on validation
rf_pred_val = rf.predict(X_valid)
etr_pred_val = etr.predict(X_valid)

# WMAE
rf_wmae = wmae(y_valid, rf_pred_val, X_valid["IsHoliday"].values)
etr_wmae = wmae(y_valid, etr_pred_val, X_valid["IsHoliday"].values)

ens_pred_val = 0.5 * (rf_pred_val + etr_pred_val)
ens_wmae = wmae(y_valid, ens_pred_val, X_valid["IsHoliday"].values)

print("RF val WMAE:", rf_wmae)
print("ETR val WMAE:", etr_wmae)
print("RF+ETR val WMAE:", ens_wmae)




- На валидации сравнил 5 подходов(3 базовых + LLAMA + лучшее решение через композицию RF и ETR через фичер инжениринг.
